# Google Drive | Google ドライブこのノートブックは、LangChainを`Google Drive API`に接続する手順を説明しています。> This notebook walks through connecting a LangChain to the `Google Drive API`.## Prerequisites | 前提条件1. Google Cloudのプロジェクトを新規作成するか、既存のプロジェクトを使用してください   > Create a Google Cloud project or use an existing project2. [Google Drive API](https://console.cloud.google.com/flows/enableapi?apiid=drive.googleapis.com)を有効にする   > Enable the [Google Drive API](https://console.cloud.google.com/flows/enableapi?apiid=drive.googleapis.com)3. [デスクトップアプリの認証情報を承認する](https://developers.google.com/drive/api/quickstart/python#authorize_credentials_for_a_desktop_application)   > [Authorize credentials for desktop app](https://developers.google.com/drive/api/quickstart/python#authorize_credentials_for_a_desktop_application)4. `pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib` を実行して、google-api-python-client、google-auth-httplib2、google-auth-oauthlib をアップグレードしながらインストールします。   > `pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib`## Instructions for retrieving your Google Docs data | Google Docsのデータを取得するための手順デフォルトでは、`GoogleDriveTools`と`GoogleDriveWrapper`は`credentials.json`ファイルが`~/.credentials/credentials.json`にあることを想定していますが、これは`GOOGLE_ACCOUNT_FILE`環境変数を使用して変更可能です。`token.json`の場所も同じディレクトリを使用します（または`token_path`パラメータを使用します）。`token.json`は、ツールを初めて使用する際に自動的に作成されることに注意してください。> By default, the `GoogleDriveTools` and `GoogleDriveWrapper` expects the `credentials.json` file to be `~/.credentials/credentials.json`, but this is configurable using the `GOOGLE_ACCOUNT_FILE` environment variable.> The location of `token.json` use the same directory (or use the parameter `token_path`). Note that `token.json` will be created automatically the first time you use the tool.`GoogleDriveSearchTool`は、いくつかのリクエストによって一連のファイルを取得することができます。> `GoogleDriveSearchTool` can retrieve a selection of files with some requests.デフォルトでは、`folder_id`を使用すると、そのフォルダ内のファイルはクエリに名前が一致した場合に`Document`として取得できます。> By default, If you use a `folder_id`, all the files inside this folder can be retrieved to `Document`, if the name match the query.

In [ ]:
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

フォルダとドキュメントのIDは、URLから取得することができます。> You can obtain your folder and document id from the URL:* フォルダ: https://drive.google.com/drive/u/0/folders/1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5 -> フォルダIDは `"1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5"` です  > Folder: https://drive.google.com/drive/u/0/folders/1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5 -> folder id is `"1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5"`* ドキュメント: https://docs.google.com/document/d/1bfaMQ18\_i56204VaQDVeAFpqEijJTgvurupdEDiaUQw/edit -> ドキュメントIDは「1bfaMQ18\_i56204VaQDVeAFpqEijJTgvurupdEDiaUQw」です。  > Document: https://docs.google.com/document/d/1bfaMQ18\_i56204VaQDVeAFpqEijJTgvurupdEDiaUQw/edit -> document id is `"1bfaMQ18_i56204VaQDVeAFpqEijJTgvurupdEDiaUQw"``root` という特別な値は、あなたの個人用ホームディレクトリのためのものです。> The special value `root` is for your personal home.

In [ ]:
folder_id = "root"
# folder_id='1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5'

デフォルトでは、これらのmime-typeを持つすべてのファイルは`Document`に変換可能です。> By default, all files with these mime-type can be converted to `Document`.* テキスト/テキスト  > text/text* オリジナル: text/plain  > text/plain* text/html  > text/html* text/csv  > text/csv* <基準>  オリジナル:  text/markdown  > text/markdown* image/png  > image/png* image/jpeg  > image/jpeg* application/epub+zip  > application/epub+zip* application/pdf  > application/pdf* application/rtf  > application/rtf* application/vnd.google-apps.document (GDoc)  > application/vnd.google-apps.document (GDoc)* application/vnd.google-apps.presentation (Google スライド)  > application/vnd.google-apps.presentation (GSlide)* application/vnd.google-apps.spreadsheet (GSheet)  > application/vnd.google-apps.spreadsheet (GSheet)* application/vnd.google.colaboratory（ノートブック colab）  > application/vnd.google.colaboratory (Notebook colab)* application/vnd.openxmlformats-officedocument.presentationml.presentation (PPTX)  > application/vnd.openxmlformats-officedocument.presentationml.presentation (PPTX)* application/vnd.openxmlformats-officedocument.wordprocessingml.document (DOCX)  > application/vnd.openxmlformats-officedocument.wordprocessingml.document (DOCX)これを更新またはカスタマイズすることが可能です。`GoogleDriveAPIWrapper`のドキュメントをご覧ください。> It's possible to update or customize this. See the documentation of `GoogleDriveAPIWrapper`.ただし、対応するパッケージがインストールされている必要があります。> But, the corresponding packages must installed.

In [ ]:
#!pip install unstructured

In [ ]:
from langchain.tools.google_drive.tool import GoogleDriveSearchTool
from langchain.utilities.google_drive import GoogleDriveAPIWrapper

# By default, search only in the filename.
tool = GoogleDriveSearchTool(
    api_wrapper=GoogleDriveAPIWrapper(
        folder_id=folder_id,
        num_results=2,
        template="gdrive-query-in-folder",  # Search in the body of documents
    )
)

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
tool.run("machine learning")

In [ ]:
tool.description

In [ ]:
from langchain.agents import load_tools

tools = load_tools(
    ["google-drive-search"],
    folder_id=folder_id,
    template="gdrive-query-in-folder",
)

## Use within an Agent | エージェント内での使用

In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
)

In [ ]:
agent.run("Search in google drive, who is 'Yann LeCun' ?")